# Day 5: Robust Portfolio Optimization

## Week 18 - Portfolio Optimization

**Date:** January 23, 2026

---

## Learning Objectives

1. **Understand estimation error** in portfolio optimization
2. **Implement shrinkage estimators** for covariance matrices
3. **Build uncertainty sets** for robust optimization
4. **Compare robust vs. classical portfolios** out-of-sample

---

## Why Robust Optimization?

Classical Mean-Variance optimization suffers from:
- **Estimation error amplification**: Small errors in inputs → Large errors in weights
- **Extreme weights**: Optimizer exploits estimation errors
- **Poor out-of-sample performance**: Optimized portfolios often underperform naive strategies

Robust optimization addresses these by:
- Acknowledging parameter uncertainty
- Optimizing for worst-case scenarios within uncertainty sets
- Shrinking extreme estimates toward structured targets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
import yfinance as yf
from typing import Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print("Robust Portfolio Optimization - Setup Complete")

---

## Part 1: Data Preparation and Estimation Error Analysis

In [ ]:
# Download market data
tickers = ['SPY', 'QQQ', 'IWM', 'EFA', 'EEM', 'TLT', 'GLD', 'VNQ', 'LQD', 'HYG']
start_date = '2018-01-01'
end_date = '2025-12-31'

print(f"Downloading data for {len(tickers)} assets...")
data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
returns = data.pct_change().dropna()

print(f"Data shape: {returns.shape}")
print(f"Date range: {returns.index[0].date()} to {returns.index[-1].date()}")
returns.head()

In [ ]:
# Demonstrate estimation error with bootstrap analysis
def bootstrap_parameters(returns: pd.DataFrame, n_bootstrap: int = 500) -> Dict:
    """
    Bootstrap estimation to quantify parameter uncertainty.
    
    Returns distribution of estimated means, volatilities, and correlations.
    """
    n_obs = len(returns)
    n_assets = returns.shape[1]
    
    means_boot = np.zeros((n_bootstrap, n_assets))
    vols_boot = np.zeros((n_bootstrap, n_assets))
    sharpe_boot = np.zeros((n_bootstrap, n_assets))
    
    for i in range(n_bootstrap):
        # Sample with replacement
        sample_idx = np.random.choice(n_obs, size=n_obs, replace=True)
        sample_returns = returns.iloc[sample_idx]
        
        means_boot[i] = sample_returns.mean().values * 252
        vols_boot[i] = sample_returns.std().values * np.sqrt(252)
        sharpe_boot[i] = means_boot[i] / vols_boot[i]
    
    return {
        'means': means_boot,
        'vols': vols_boot,
        'sharpe': sharpe_boot
    }

# Run bootstrap
boot_results = bootstrap_parameters(returns)

# Display estimation uncertainty
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Mean returns uncertainty
ax = axes[0]
mean_means = boot_results['means'].mean(axis=0)
mean_stds = boot_results['means'].std(axis=0)
x = np.arange(len(tickers))
ax.bar(x, mean_means, yerr=2*mean_stds, capsize=3, alpha=0.7, color='steelblue')
ax.set_xticks(x)
ax.set_xticklabels(tickers, rotation=45)
ax.set_ylabel('Annualized Return')
ax.set_title('Mean Returns with 95% CI')
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)

# Sharpe ratio uncertainty
ax = axes[1]
sharpe_means = boot_results['sharpe'].mean(axis=0)
sharpe_stds = boot_results['sharpe'].std(axis=0)
ax.bar(x, sharpe_means, yerr=2*sharpe_stds, capsize=3, alpha=0.7, color='darkgreen')
ax.set_xticks(x)
ax.set_xticklabels(tickers, rotation=45)
ax.set_ylabel('Sharpe Ratio')
ax.set_title('Sharpe Ratios with 95% CI')
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)

# Coefficient of variation for means (relative uncertainty)
ax = axes[2]
cv = np.abs(mean_stds / (mean_means + 1e-8))
ax.bar(x, cv, alpha=0.7, color='coral')
ax.set_xticks(x)
ax.set_xticklabels(tickers, rotation=45)
ax.set_ylabel('CV (Std/Mean)')
ax.set_title('Relative Estimation Uncertainty')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("KEY INSIGHT: Expected returns have VERY high estimation error")
print("Covariance estimates are more stable than mean estimates")
print("="*60)

---

## Part 2: Shrinkage Estimators for Covariance Matrices

### The Shrinkage Framework

Shrinkage estimator: $\hat{\Sigma}_{shrink} = \delta \cdot F + (1-\delta) \cdot S$

Where:
- $S$ = Sample covariance matrix
- $F$ = Structured target (shrinkage target)
- $\delta$ = Shrinkage intensity (0 to 1)

In [ ]:
class CovarianceShrinkage:
    """
    Collection of covariance shrinkage estimators.
    
    Implements:
    1. Ledoit-Wolf shrinkage to identity
    2. Ledoit-Wolf shrinkage to constant correlation
    3. Oracle Approximating Shrinkage (OAS)
    """
    
    def __init__(self, returns: pd.DataFrame):
        self.returns = returns
        self.T, self.N = returns.shape
        self.sample_cov = returns.cov().values
        
    def ledoit_wolf_identity(self) -> Tuple[np.ndarray, float]:
        """
        Ledoit-Wolf shrinkage toward scaled identity matrix.
        
        Target: F = trace(S)/N * I
        """
        X = self.returns.values
        T, N = self.T, self.N
        
        # Sample covariance
        S = self.sample_cov
        
        # Target: scaled identity
        mu = np.trace(S) / N
        F = mu * np.eye(N)
        
        # Compute optimal shrinkage intensity
        # Using Ledoit-Wolf formula
        X_centered = X - X.mean(axis=0)
        
        # Sum of squared off-diagonal elements
        delta = S - F
        
        # pi: sum of squared differences from sample to target
        pi = np.sum(delta ** 2)
        
        # rho: asymptotic sum of squared shrinkage residuals
        y = X_centered ** 2
        phi_diag = np.sum((y.T @ y / T - S ** 2)) / T
        
        rho = 0
        for i in range(N):
            for j in range(N):
                if i == j:
                    rho += np.var(y[:, i]) / T
                else:
                    rho += np.cov(y[:, i], y[:, j])[0, 1] / T
        
        # Optimal shrinkage
        kappa = (pi - rho) / pi if pi > 0 else 0
        delta_star = max(0, min(1, kappa / T))
        
        # Shrunk covariance
        shrunk_cov = delta_star * F + (1 - delta_star) * S
        
        return shrunk_cov, delta_star
    
    def ledoit_wolf_constant_correlation(self) -> Tuple[np.ndarray, float]:
        """
        Ledoit-Wolf shrinkage toward constant correlation matrix.
        
        Target: F_ij = sqrt(S_ii * S_jj) * rho_bar for i != j
        """
        X = self.returns.values
        T, N = self.T, self.N
        
        # Sample covariance and correlation
        S = self.sample_cov
        vols = np.sqrt(np.diag(S))
        corr = S / np.outer(vols, vols)
        np.fill_diagonal(corr, 1.0)
        
        # Average correlation (excluding diagonal)
        rho_bar = (np.sum(corr) - N) / (N * (N - 1))
        
        # Target matrix: constant correlation
        F_corr = np.full((N, N), rho_bar)
        np.fill_diagonal(F_corr, 1.0)
        F = np.outer(vols, vols) * F_corr
        
        # Compute shrinkage intensity (simplified)
        X_centered = X - X.mean(axis=0)
        X_std = X_centered / vols
        
        # Estimate pi (sum of squared errors)
        pi = np.sum((S - F) ** 2)
        
        # Simple approximation for optimal shrinkage
        gamma = np.sum((corr - rho_bar) ** 2) - np.sum((corr - rho_bar).diagonal() ** 2)
        kappa = gamma / (T * pi) if pi > 0 else 0
        delta_star = max(0, min(1, kappa))
        
        # Shrunk covariance
        shrunk_cov = delta_star * F + (1 - delta_star) * S
        
        return shrunk_cov, delta_star
    
    def oracle_approximating_shrinkage(self) -> Tuple[np.ndarray, float]:
        """
        Oracle Approximating Shrinkage (OAS) estimator.
        
        More efficient than Ledoit-Wolf for small samples.
        """
        T, N = self.T, self.N
        S = self.sample_cov
        
        # Target: scaled identity
        mu = np.trace(S) / N
        F = mu * np.eye(N)
        
        # OAS shrinkage intensity
        trace_S2 = np.trace(S @ S)
        trace_S_2 = np.trace(S) ** 2
        
        # OAS formula
        rho_num = (1 - 2/N) * trace_S2 + trace_S_2
        rho_den = (T + 1 - 2/N) * (trace_S2 - trace_S_2 / N)
        
        delta_star = max(0, min(1, rho_num / rho_den if rho_den > 0 else 0))
        
        # Shrunk covariance
        shrunk_cov = delta_star * F + (1 - delta_star) * S
        
        return shrunk_cov, delta_star


# Compute shrinkage estimators
shrinkage = CovarianceShrinkage(returns)

cov_lw_identity, delta_identity = shrinkage.ledoit_wolf_identity()
cov_lw_constcorr, delta_constcorr = shrinkage.ledoit_wolf_constant_correlation()
cov_oas, delta_oas = shrinkage.oracle_approximating_shrinkage()
cov_sample = returns.cov().values

print("Shrinkage Intensities:")
print(f"  Ledoit-Wolf (Identity):      δ = {delta_identity:.4f}")
print(f"  Ledoit-Wolf (Const Corr):    δ = {delta_constcorr:.4f}")
print(f"  OAS:                         δ = {delta_oas:.4f}")

In [ ]:
# Visualize covariance matrices
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

cov_matrices = {
    'Sample Covariance': cov_sample,
    'LW Shrink to Identity': cov_lw_identity,
    'LW Shrink to Const Corr': cov_lw_constcorr,
    'OAS': cov_oas
}

# Convert to correlation for visualization
for ax, (name, cov) in zip(axes.flat, cov_matrices.items()):
    vols = np.sqrt(np.diag(cov))
    corr = cov / np.outer(vols, vols)
    
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                xticklabels=tickers, yticklabels=tickers, ax=ax,
                vmin=-1, vmax=1)
    ax.set_title(f'{name}\n(as correlation matrix)')

plt.tight_layout()
plt.show()

In [ ]:
# Compare eigenvalue distributions (condition numbers)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Eigenvalues
ax = axes[0]
for name, cov in cov_matrices.items():
    eigenvalues = np.linalg.eigvalsh(cov)
    eigenvalues = np.sort(eigenvalues)[::-1]
    ax.semilogy(range(1, len(eigenvalues) + 1), eigenvalues, 'o-', label=name, markersize=8)

ax.set_xlabel('Eigenvalue Rank')
ax.set_ylabel('Eigenvalue (log scale)')
ax.set_title('Eigenvalue Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Condition numbers
ax = axes[1]
condition_numbers = []
for name, cov in cov_matrices.items():
    cond = np.linalg.cond(cov)
    condition_numbers.append((name, cond))

names, conds = zip(*condition_numbers)
colors = ['coral', 'steelblue', 'darkgreen', 'purple']
bars = ax.bar(range(len(names)), conds, color=colors, alpha=0.7)
ax.set_xticks(range(len(names)))
ax.set_xticklabels(['Sample', 'LW-Identity', 'LW-ConstCorr', 'OAS'], rotation=15)
ax.set_ylabel('Condition Number')
ax.set_title('Matrix Condition Numbers (lower = more stable)')

for bar, cond in zip(bars, conds):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{cond:.1f}',
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("KEY INSIGHT: Shrinkage reduces condition number")
print("Lower condition number = more numerically stable optimization")
print("="*60)

---

## Part 3: Robust Optimization with Uncertainty Sets

### Uncertainty Set Framework

Instead of point estimates, we define uncertainty sets:

**Mean uncertainty:** $\mu \in \mathcal{U}_\mu = \{\mu : \|\mu - \hat{\mu}\| \leq \kappa_\mu\}$

**Covariance uncertainty:** $\Sigma \in \mathcal{U}_\Sigma$

The robust problem becomes:

$$\max_w \min_{\mu \in \mathcal{U}_\mu} w^T \mu - \frac{\lambda}{2} w^T \Sigma w$$

In [ ]:
class RobustPortfolioOptimizer:
    """
    Robust portfolio optimization with uncertainty sets.
    
    Implements:
    1. Classical Mean-Variance (benchmark)
    2. Robust Mean optimization (box uncertainty)
    3. Robust Mean optimization (ellipsoidal uncertainty)
    4. Resampled optimization
    """
    
    def __init__(self, returns: pd.DataFrame, risk_aversion: float = 2.0):
        self.returns = returns
        self.mu = returns.mean().values * 252  # Annualized
        self.Sigma = returns.cov().values * 252  # Annualized
        self.N = len(returns.columns)
        self.T = len(returns)
        self.risk_aversion = risk_aversion
        self.asset_names = returns.columns.tolist()
        
        # Estimate uncertainty
        self._estimate_uncertainty()
    
    def _estimate_uncertainty(self):
        """Estimate parameter uncertainty from data."""
        # Standard error of mean estimates
        self.mu_se = np.sqrt(np.diag(self.Sigma) / self.T)
        
        # Box uncertainty: 2 standard errors
        self.mu_box_radius = 2 * self.mu_se
        
        # Ellipsoidal uncertainty: use inverse covariance
        # Scaled by chi-squared critical value
        chi2_crit = stats.chi2.ppf(0.95, self.N)
        self.ellipsoid_radius = np.sqrt(chi2_crit / self.T)
    
    def classical_mean_variance(self, target_return: Optional[float] = None) -> np.ndarray:
        """
        Classical Mean-Variance optimization.
        
        max w'μ - (λ/2) w'Σw
        s.t. sum(w) = 1
        """
        def objective(w):
            port_return = w @ self.mu
            port_var = w @ self.Sigma @ w
            return -(port_return - 0.5 * self.risk_aversion * port_var)
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        
        if target_return is not None:
            constraints.append({'type': 'eq', 'fun': lambda w: w @ self.mu - target_return})
        
        bounds = [(0, 1) for _ in range(self.N)]  # Long-only
        w0 = np.ones(self.N) / self.N
        
        result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
        return result.x
    
    def robust_box_uncertainty(self) -> np.ndarray:
        """
        Robust optimization with box uncertainty set.
        
        Worst-case mean: μ_wc = μ - sign(w) * κ
        
        For long-only, worst-case is always: μ_wc = μ - κ
        """
        def objective(w):
            # Worst-case return (subtract uncertainty for long positions)
            worst_case_mu = self.mu - self.mu_box_radius * np.sign(w + 1e-8)
            port_return = w @ worst_case_mu
            port_var = w @ self.Sigma @ w
            return -(port_return - 0.5 * self.risk_aversion * port_var)
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0, 1) for _ in range(self.N)]
        w0 = np.ones(self.N) / self.N
        
        result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
        return result.x
    
    def robust_ellipsoidal_uncertainty(self) -> np.ndarray:
        """
        Robust optimization with ellipsoidal uncertainty set.
        
        Uncertainty set: ||Σ^(-1/2)(μ - μ̂)||₂ ≤ κ
        
        Worst-case return: w'μ̂ - κ * ||Σ^(1/2)w||₂
        """
        # Compute Sigma^(1/2) for penalty term
        eigenvalues, eigenvectors = np.linalg.eigh(self.Sigma)
        Sigma_sqrt = eigenvectors @ np.diag(np.sqrt(np.maximum(eigenvalues, 0))) @ eigenvectors.T
        
        def objective(w):
            port_return = w @ self.mu
            port_var = w @ self.Sigma @ w
            
            # Ellipsoidal penalty: reduces expected return
            uncertainty_penalty = self.ellipsoid_radius * np.sqrt(w @ self.Sigma @ w / self.T)
            
            robust_return = port_return - uncertainty_penalty
            return -(robust_return - 0.5 * self.risk_aversion * port_var)
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0, 1) for _ in range(self.N)]
        w0 = np.ones(self.N) / self.N
        
        result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
        return result.x
    
    def resampled_optimization(self, n_samples: int = 500) -> np.ndarray:
        """
        Resampled Efficient Frontier (Michaud, 1998).
        
        Bootstrap parameters and average optimal weights.
        """
        weights_samples = np.zeros((n_samples, self.N))
        
        for i in range(n_samples):
            # Bootstrap returns
            sample_idx = np.random.choice(self.T, size=self.T, replace=True)
            sample_returns = self.returns.iloc[sample_idx]
            
            # Estimate parameters from bootstrap sample
            mu_boot = sample_returns.mean().values * 252
            Sigma_boot = sample_returns.cov().values * 252
            
            # Optimize with bootstrap parameters
            def objective(w):
                port_return = w @ mu_boot
                port_var = w @ Sigma_boot @ w
                return -(port_return - 0.5 * self.risk_aversion * port_var)
            
            constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
            bounds = [(0, 1) for _ in range(self.N)]
            w0 = np.ones(self.N) / self.N
            
            try:
                result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
                weights_samples[i] = result.x
            except:
                weights_samples[i] = w0
        
        # Average weights
        return weights_samples.mean(axis=0)
    
    def minimum_variance(self, use_shrinkage: bool = False) -> np.ndarray:
        """
        Global Minimum Variance portfolio.
        
        Ignores expected returns entirely.
        """
        if use_shrinkage:
            shrinkage = CovarianceShrinkage(self.returns)
            Sigma, _ = shrinkage.ledoit_wolf_constant_correlation()
            Sigma = Sigma * 252
        else:
            Sigma = self.Sigma
        
        def objective(w):
            return w @ Sigma @ w
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0, 1) for _ in range(self.N)]
        w0 = np.ones(self.N) / self.N
        
        result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
        return result.x


# Create optimizer
optimizer = RobustPortfolioOptimizer(returns, risk_aversion=2.0)

print("RobustPortfolioOptimizer initialized")
print(f"Assets: {optimizer.asset_names}")
print(f"Observations: {optimizer.T}")

In [ ]:
# Compute all portfolio weights
print("Computing portfolios...")

portfolios = {
    'Equal Weight': np.ones(optimizer.N) / optimizer.N,
    'Classical MV': optimizer.classical_mean_variance(),
    'Min Variance': optimizer.minimum_variance(use_shrinkage=False),
    'Min Var (Shrunk)': optimizer.minimum_variance(use_shrinkage=True),
    'Robust Box': optimizer.robust_box_uncertainty(),
    'Robust Ellipsoid': optimizer.robust_ellipsoidal_uncertainty(),
    'Resampled': optimizer.resampled_optimization(n_samples=200)
}

# Create weights DataFrame
weights_df = pd.DataFrame(portfolios, index=optimizer.asset_names).T
print("\nPortfolio Weights:")
print(weights_df.round(4))

In [ ]:
# Visualize portfolio weights
fig, ax = plt.subplots(figsize=(14, 8))

weights_df.plot(kind='bar', ax=ax, colormap='tab10', width=0.8)
ax.set_ylabel('Weight')
ax.set_xlabel('Portfolio Strategy')
ax.set_title('Portfolio Weights Comparison: Classical vs Robust Methods')
ax.legend(title='Assets', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

# Weight concentration metrics
print("\nWeight Concentration Metrics:")
print("="*60)
for name, weights in portfolios.items():
    hhi = np.sum(weights ** 2)  # Herfindahl-Hirschman Index
    max_weight = np.max(weights)
    n_effective = 1 / hhi  # Effective number of assets
    print(f"{name:20s}: HHI={hhi:.4f}, MaxWt={max_weight:.4f}, N_eff={n_effective:.2f}")

---

## Part 4: Out-of-Sample Performance Evaluation

In [ ]:
def rolling_backtest(returns: pd.DataFrame, 
                     estimation_window: int = 252,
                     rebalance_freq: int = 21,
                     risk_aversion: float = 2.0) -> Dict[str, pd.Series]:
    """
    Rolling window backtest comparing robust vs classical portfolios.
    
    Parameters:
    -----------
    returns : Daily returns DataFrame
    estimation_window : Days for parameter estimation
    rebalance_freq : Days between rebalancing
    risk_aversion : Risk aversion parameter
    
    Returns:
    --------
    Dictionary of portfolio return series
    """
    n_obs = len(returns)
    N = returns.shape[1]
    
    # Initialize tracking
    portfolio_returns = {name: [] for name in ['Equal Weight', 'Classical MV', 
                                                'Min Variance', 'Robust Box', 'Resampled']}
    dates = []
    
    # Current weights
    current_weights = {name: np.ones(N) / N for name in portfolio_returns.keys()}
    
    for t in range(estimation_window, n_obs):
        # Rebalance at specified frequency
        if (t - estimation_window) % rebalance_freq == 0:
            # Estimation window
            est_returns = returns.iloc[t-estimation_window:t]
            
            try:
                optimizer = RobustPortfolioOptimizer(est_returns, risk_aversion)
                
                current_weights['Equal Weight'] = np.ones(N) / N
                current_weights['Classical MV'] = optimizer.classical_mean_variance()
                current_weights['Min Variance'] = optimizer.minimum_variance(use_shrinkage=True)
                current_weights['Robust Box'] = optimizer.robust_box_uncertainty()
                current_weights['Resampled'] = optimizer.resampled_optimization(n_samples=100)
            except Exception as e:
                pass  # Keep previous weights if optimization fails
        
        # Record returns for this period
        daily_return = returns.iloc[t].values
        dates.append(returns.index[t])
        
        for name, weights in current_weights.items():
            port_ret = weights @ daily_return
            portfolio_returns[name].append(port_ret)
    
    # Convert to Series
    return {name: pd.Series(rets, index=dates) 
            for name, rets in portfolio_returns.items()}


# Run backtest
print("Running rolling backtest (this may take a minute)...")
backtest_results = rolling_backtest(returns, estimation_window=252, rebalance_freq=21)
print("Backtest complete!")

In [ ]:
# Performance analysis
def calculate_metrics(returns_series: pd.Series) -> Dict:
    """Calculate comprehensive performance metrics."""
    ann_return = returns_series.mean() * 252
    ann_vol = returns_series.std() * np.sqrt(252)
    sharpe = ann_return / ann_vol if ann_vol > 0 else 0
    
    # Drawdown
    cum_returns = (1 + returns_series).cumprod()
    rolling_max = cum_returns.cummax()
    drawdowns = (cum_returns - rolling_max) / rolling_max
    max_dd = drawdowns.min()
    
    # Sortino ratio
    downside_returns = returns_series[returns_series < 0]
    downside_vol = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
    sortino = ann_return / downside_vol if downside_vol > 0 else 0
    
    # Calmar ratio
    calmar = -ann_return / max_dd if max_dd < 0 else 0
    
    return {
        'Annual Return': ann_return,
        'Annual Vol': ann_vol,
        'Sharpe': sharpe,
        'Sortino': sortino,
        'Max Drawdown': max_dd,
        'Calmar': calmar
    }

# Calculate metrics for all portfolios
metrics_df = pd.DataFrame({name: calculate_metrics(rets) 
                           for name, rets in backtest_results.items()}).T

print("\nOut-of-Sample Performance Metrics:")
print("="*80)
print(metrics_df.round(4))

In [ ]:
# Visualize backtest results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Cumulative returns
ax = axes[0, 0]
for name, rets in backtest_results.items():
    cum_rets = (1 + rets).cumprod()
    ax.plot(cum_rets.index, cum_rets.values, label=name, linewidth=1.5)
ax.set_ylabel('Cumulative Return')
ax.set_title('Out-of-Sample Cumulative Returns')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# Rolling Sharpe ratio (252-day)
ax = axes[0, 1]
for name, rets in backtest_results.items():
    rolling_sharpe = (rets.rolling(252).mean() * 252) / (rets.rolling(252).std() * np.sqrt(252))
    ax.plot(rolling_sharpe.index, rolling_sharpe.values, label=name, linewidth=1.5)
ax.set_ylabel('Rolling Sharpe (252d)')
ax.set_title('Rolling Sharpe Ratio')
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# Drawdowns
ax = axes[1, 0]
for name, rets in backtest_results.items():
    cum_rets = (1 + rets).cumprod()
    rolling_max = cum_rets.cummax()
    drawdowns = (cum_rets - rolling_max) / rolling_max
    ax.fill_between(drawdowns.index, drawdowns.values, 0, alpha=0.3, label=name)
ax.set_ylabel('Drawdown')
ax.set_title('Underwater Curve (Drawdowns)')
ax.legend(loc='lower left')
ax.grid(True, alpha=0.3)

# Performance metrics bar chart
ax = axes[1, 1]
metrics_plot = metrics_df[['Sharpe', 'Sortino', 'Calmar']]
metrics_plot.plot(kind='bar', ax=ax, colormap='viridis', width=0.7)
ax.set_ylabel('Ratio')
ax.set_title('Risk-Adjusted Performance Comparison')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.legend(loc='upper right')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

---

## Part 5: Black-Litterman with Uncertainty

Black-Litterman naturally incorporates uncertainty through investor views.

In [ ]:
class BlackLittermanRobust:
    """
    Black-Litterman model with explicit uncertainty handling.
    
    The model naturally blends market equilibrium with uncertain views.
    """
    
    def __init__(self, returns: pd.DataFrame, market_caps: Optional[np.ndarray] = None,
                 risk_aversion: float = 2.5, tau: float = 0.05):
        self.returns = returns
        self.N = returns.shape[1]
        self.asset_names = returns.columns.tolist()
        
        # Use shrunk covariance
        shrinkage = CovarianceShrinkage(returns)
        self.Sigma, _ = shrinkage.ledoit_wolf_constant_correlation()
        self.Sigma = self.Sigma * 252  # Annualize
        
        # Market caps (default: equal)
        self.market_caps = market_caps if market_caps is not None else np.ones(self.N)
        self.w_mkt = self.market_caps / np.sum(self.market_caps)
        
        self.risk_aversion = risk_aversion
        self.tau = tau
        
        # Implied equilibrium returns
        self.pi = risk_aversion * self.Sigma @ self.w_mkt
    
    def add_views(self, P: np.ndarray, Q: np.ndarray, 
                  confidences: Optional[np.ndarray] = None) -> np.ndarray:
        """
        Incorporate investor views into posterior expected returns.
        
        Parameters:
        -----------
        P : View matrix (K x N), K views on N assets
        Q : View returns (K,), expected returns for each view
        confidences : View confidences (K,), 0 to 1 scale
        
        Returns:
        --------
        Posterior expected returns (N,)
        """
        K = len(Q)  # Number of views
        
        # Omega: view uncertainty covariance (diagonal)
        if confidences is None:
            confidences = np.ones(K) * 0.5
        
        # Scale omega based on confidence (lower confidence = higher uncertainty)
        omega_diag = (1 / confidences - 1) * np.diag(P @ self.Sigma @ P.T) * self.tau
        Omega = np.diag(omega_diag)
        
        # Prior covariance of expected returns
        tau_Sigma = self.tau * self.Sigma
        
        # Posterior expected returns (Black-Litterman formula)
        inv_tau_Sigma = np.linalg.inv(tau_Sigma)
        inv_Omega = np.linalg.inv(Omega)
        
        M = np.linalg.inv(inv_tau_Sigma + P.T @ inv_Omega @ P)
        posterior_mu = M @ (inv_tau_Sigma @ self.pi + P.T @ inv_Omega @ Q)
        
        return posterior_mu
    
    def optimize(self, mu: np.ndarray) -> np.ndarray:
        """
        Optimize portfolio given expected returns.
        """
        def objective(w):
            return -(w @ mu - 0.5 * self.risk_aversion * w @ self.Sigma @ w)
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0, 1) for _ in range(self.N)]
        w0 = self.w_mkt.copy()
        
        result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
        return result.x


# Example: Black-Litterman with views
bl_model = BlackLittermanRobust(returns, risk_aversion=2.5, tau=0.05)

print("Implied Equilibrium Returns (from market portfolio):")
for name, ret in zip(bl_model.asset_names, bl_model.pi):
    print(f"  {name}: {ret*100:.2f}%")

In [ ]:
# Add investor views
# View 1: QQQ will outperform SPY by 3%
# View 2: EEM will underperform EFA by 2%
# View 3: TLT will return 4%

K = 3  # Number of views
N = len(tickers)

# Build view matrix P
P = np.zeros((K, N))

# View 1: QQQ - SPY = 3%
P[0, tickers.index('QQQ')] = 1
P[0, tickers.index('SPY')] = -1

# View 2: EFA - EEM = 2%
P[1, tickers.index('EFA')] = 1
P[1, tickers.index('EEM')] = -1

# View 3: TLT absolute = 4%
P[2, tickers.index('TLT')] = 1

# View returns
Q = np.array([0.03, 0.02, 0.04])

# Confidences (higher = more confident)
confidences = np.array([0.6, 0.4, 0.7])

# Compute posterior returns
posterior_mu = bl_model.add_views(P, Q, confidences)

# Compare prior (equilibrium) vs posterior
print("\nView Incorporation Results:")
print("="*60)
print(f"{'Asset':<10} {'Equilibrium':>12} {'Posterior':>12} {'Change':>10}")
print("-"*60)
for i, name in enumerate(tickers):
    change = (posterior_mu[i] - bl_model.pi[i]) * 100
    print(f"{name:<10} {bl_model.pi[i]*100:>11.2f}% {posterior_mu[i]*100:>11.2f}% {change:>+9.2f}%")

In [ ]:
# Optimize portfolios: prior vs posterior
w_prior = bl_model.optimize(bl_model.pi)
w_posterior = bl_model.optimize(posterior_mu)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Returns comparison
ax = axes[0]
x = np.arange(N)
width = 0.35
ax.bar(x - width/2, bl_model.pi * 100, width, label='Equilibrium', color='steelblue', alpha=0.7)
ax.bar(x + width/2, posterior_mu * 100, width, label='Posterior (with views)', color='coral', alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels(tickers, rotation=45)
ax.set_ylabel('Expected Return (%)')
ax.set_title('Black-Litterman: Prior vs Posterior Returns')
ax.legend()
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)

# Weights comparison
ax = axes[1]
ax.bar(x - width/2, w_prior * 100, width, label='Equilibrium Portfolio', color='steelblue', alpha=0.7)
ax.bar(x + width/2, w_posterior * 100, width, label='Posterior Portfolio', color='coral', alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels(tickers, rotation=45)
ax.set_ylabel('Weight (%)')
ax.set_title('Black-Litterman: Portfolio Weights')
ax.legend()

plt.tight_layout()
plt.show()

---

## Part 6: Interview Questions and Key Concepts

### Common Interview Questions

**Q1: Why does classical Mean-Variance optimization often perform poorly out-of-sample?**

**A1:** Classical MV optimization suffers from:
1. **Estimation error amplification**: The optimizer treats estimated parameters as exact truth and exploits estimation errors
2. **Extreme positions**: Takes large positions in assets with overestimated returns or underestimated correlations
3. **Instability**: Small changes in inputs lead to large weight changes
4. **High condition number**: Sample covariance may be nearly singular with few observations

---

**Q2: Explain the Ledoit-Wolf shrinkage estimator.**

**A2:** Ledoit-Wolf shrinks the sample covariance toward a structured target:

$\hat{\Sigma} = \delta F + (1-\delta)S$

Where:
- $S$ = Sample covariance (unbiased but high variance)
- $F$ = Target (biased but low variance), e.g., scaled identity or constant correlation
- $\delta$ = Optimal shrinkage intensity (derived analytically)

The optimal $\delta$ minimizes the expected Frobenius norm between the true and estimated covariance matrices.

---

**Q3: What are uncertainty sets in robust optimization?**

**A3:** Uncertainty sets define regions where true parameters might lie:

- **Box uncertainty**: $\mu_i \in [\hat{\mu}_i - \kappa_i, \hat{\mu}_i + \kappa_i]$
- **Ellipsoidal uncertainty**: $\|\Sigma^{-1/2}(\mu - \hat{\mu})\|_2 \leq \kappa$

The robust problem optimizes for the worst case within the uncertainty set, leading to more conservative but stable portfolios.

---

**Q4: How does Black-Litterman handle estimation uncertainty?**

**A4:** Black-Litterman naturally incorporates uncertainty through:

1. **Prior**: Equilibrium returns from market portfolio (stable anchor)
2. **Views**: Investor opinions with explicit uncertainty (Ω matrix)
3. **Posterior**: Bayesian combination weighted by relative confidences

The τ parameter controls prior uncertainty, while Ω controls view uncertainty. Lower confidence views have less impact on the posterior.

In [ ]:
# Demonstrate sensitivity analysis: how weights change with small parameter changes

def sensitivity_analysis(returns: pd.DataFrame, n_perturbations: int = 50):
    """
    Analyze weight sensitivity to small parameter perturbations.
    """
    base_mu = returns.mean().values * 252
    base_Sigma = returns.cov().values * 252
    N = len(base_mu)
    
    # Store weight variations
    classical_weights = []
    robust_weights = []
    
    for _ in range(n_perturbations):
        # Perturb means by small amount (10% of standard error)
        mu_perturb = base_mu + np.random.randn(N) * 0.01
        
        # Classical optimization
        def obj_classical(w):
            return -(w @ mu_perturb - 0.5 * 2.0 * w @ base_Sigma @ w)
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0, 1) for _ in range(N)]
        w0 = np.ones(N) / N
        
        res = minimize(obj_classical, w0, method='SLSQP', bounds=bounds, constraints=constraints)
        classical_weights.append(res.x)
        
        # Robust optimization (with penalty for uncertainty)
        def obj_robust(w):
            penalty = 0.1 * np.sqrt(w @ base_Sigma @ w)
            return -(w @ mu_perturb - penalty - 0.5 * 2.0 * w @ base_Sigma @ w)
        
        res = minimize(obj_robust, w0, method='SLSQP', bounds=bounds, constraints=constraints)
        robust_weights.append(res.x)
    
    return np.array(classical_weights), np.array(robust_weights)

# Run sensitivity analysis
classical_w, robust_w = sensitivity_analysis(returns)

# Visualize weight stability
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
classical_std = classical_w.std(axis=0)
robust_std = robust_w.std(axis=0)

x = np.arange(len(tickers))
width = 0.35
ax.bar(x - width/2, classical_std * 100, width, label='Classical MV', color='coral', alpha=0.7)
ax.bar(x + width/2, robust_std * 100, width, label='Robust', color='steelblue', alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels(tickers, rotation=45)
ax.set_ylabel('Weight Std Dev (%)')
ax.set_title('Weight Stability: Std Dev Under Parameter Perturbation')
ax.legend()

# Box plot of weights
ax = axes[1]
positions = np.arange(len(tickers)) * 2
bp1 = ax.boxplot(classical_w * 100, positions=positions - 0.4, widths=0.6, 
                 patch_artist=True, boxprops=dict(facecolor='coral', alpha=0.7))
bp2 = ax.boxplot(robust_w * 100, positions=positions + 0.4, widths=0.6,
                 patch_artist=True, boxprops=dict(facecolor='steelblue', alpha=0.7))
ax.set_xticks(positions)
ax.set_xticklabels(tickers, rotation=45)
ax.set_ylabel('Weight (%)')
ax.set_title('Weight Distributions Under Parameter Uncertainty')
ax.legend([bp1['boxes'][0], bp2['boxes'][0]], ['Classical MV', 'Robust'])

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("KEY INSIGHT: Robust methods produce more stable weights")
print("Classical MV weights vary significantly with small input changes")
print("="*60)

---

## Summary: Robust Portfolio Optimization

### Key Takeaways

1. **Estimation error is the enemy**: Mean returns have very high estimation uncertainty

2. **Shrinkage estimators**:
   - Reduce covariance matrix condition number
   - Trade off bias for variance reduction
   - Ledoit-Wolf provides optimal shrinkage analytically

3. **Uncertainty sets**:
   - Acknowledge parameter uncertainty explicitly
   - Box and ellipsoidal sets are most common
   - Leads to more conservative, stable portfolios

4. **Black-Litterman**:
   - Natural framework for incorporating uncertainty
   - Blends equilibrium (stable) with views (uncertain)
   - Produces intuitive, stable portfolios

5. **Practical guidance**:
   - Always use shrinkage for covariance estimation
   - Consider ignoring return estimates (min variance)
   - Resampling provides simple robustification
   - Backtest with rolling windows to assess real-world performance

---

### References

1. Ledoit, O., & Wolf, M. (2004). A Well-Conditioned Estimator for Large-Dimensional Covariance Matrices
2. Michaud, R. O. (1998). Efficient Asset Management: A Practical Guide to Stock Portfolio Optimization
3. Goldfarb, D., & Iyengar, G. (2003). Robust Portfolio Selection Problems
4. Black, F., & Litterman, R. (1992). Global Portfolio Optimization

In [ ]:
# Final summary: Best practices checklist

print("\n" + "="*70)
print("ROBUST PORTFOLIO OPTIMIZATION: BEST PRACTICES CHECKLIST")
print("="*70)

checklist = [
    "✓ Use shrinkage estimators for covariance (Ledoit-Wolf, OAS)",
    "✓ Quantify estimation uncertainty via bootstrap",
    "✓ Consider ignoring return estimates (minimum variance)",
    "✓ Apply constraints (long-only, max weight, sector limits)",
    "✓ Use resampling to average over parameter uncertainty",
    "✓ Implement Black-Litterman for view incorporation",
    "✓ Backtest with rolling windows (out-of-sample)",
    "✓ Compare against naive benchmarks (1/N, equal weight)",
    "✓ Monitor weight stability and turnover",
    "✓ Report risk-adjusted metrics (Sharpe, Sortino, Calmar)"
]

for item in checklist:
    print(f"  {item}")

print("\n" + "="*70)
print("Day 5 Complete: Robust Portfolio Optimization")
print("="*70)